# Chapter 1

In [36]:
import torch

def get_parallel_db(db, remove_idx):
    return torch.cat((db[0:remove_idx], db[remove_idx + 1:]))


def get_parallel_dbs(db):
    pdbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        pdbs.append(pdb)

    return pdbs


def create_db_and_parallels(length):
    db = torch.rand(length) > 0.5
    pdb = get_parallel_dbs(db)
    return db, pdb

In [37]:
db, pdbs = create_db_and_parallels(5000)

In [38]:
def query(db):
    return db.sum()

In [39]:
full_db_result = query(db)

In [40]:
sensitivity = 0

for pdb in pdbs:
    pdb_results = query(pdb)
    
    db_distance = torch.abs(pdb_results-full_db_result)
    
    if db_distance > sensitivity:
        sensitivity = db_distance

In [41]:
sensitivity

tensor(1)

# Chapter 3

In [42]:
def sensitivity(query, num_entries=1000):
    db, pdbs = create_db_and_parallels(num_entries)
    full_db_result = query(db)
    
    max_distance = 0
    
    for pdb in pdbs:
        pdb_results = query(pdb)

        db_distance = torch.abs(pdb_results-full_db_result)

        if db_distance > max_distance:
            max_distance = db_distance
    
    return max_distance

In [47]:
def query(db):
    
    return db.float().mean() 

In [48]:
sensitivity(query)

tensor(0.0005)

# Chapter 5

In [49]:
def query(db, threshold=5):
    return (db.sum() > threshold).float()

In [56]:
for i in range(10):
    ans = sensitivity(query, num_entries=10)
    print(ans)

0
0
tensor(1.)
0
0
tensor(1.)
0
0
tensor(1.)
0


# Chapter 7

In [66]:
db, _ = create_db_and_parallels(100)
pdb = get_parallel_db(db, 5)

In [67]:
db

tensor([1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 0, 1], dtype=torch.uint8)

In [68]:
pdb

tensor([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 1], dtype=torch.uint8)

In [72]:
sum(pdb)

tensor(54, dtype=torch.uint8)

In [70]:
# sum
sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [71]:
# mean
(sum(db).float()/len(db)) - (sum(pdb).float()/len(db))

tensor(0.)

In [75]:
# threshold

(sum(db).float() > 49) - (sum(pdb).float()>54)

tensor(1, dtype=torch.uint8)